In [7]:
#Run this cell to attach libraries
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
library(RColorBrewer)
library(themis)

ERROR: Error in library(themis): there is no package called ‘themis’


In [2]:
install.packages("themis")

also installing the dependencies ‘RANN’, ‘ROSE’


Warning message in install.packages("themis"):
“installation of package ‘RANN’ had non-zero exit status”
Warning message in install.packages("themis"):
“installation of package ‘themis’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [2]:
data <- read_delim(file = "data/data.csv", delim = ";") #Load the data into R. File is semi-colon separated so use read_delim function
head(data)

Rows: 4424 Columns: 37
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr  (1): Target
dbl (36): Marital_status, Application_mode, Application_order, Course, Dayti...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Marital_status,Application_mode,Application_order,Course,Daytime/evening attendance,Previous_qualification,Previous_qualification_grade,Nacionality,Mother_qualification,Father_qualification,⋯,Curricular_units_2nd_sem_(credited),Curricular_units_2nd_sem_(enrolled),Curricular_units_2nd_sem_(evaluations),Curricular_units_2nd_sem_(approved),Curricular_units_2nd_sem_(grade),Curricular_units_2nd_sem_(without evaluations),Unemployment_rate,Inflation_rate,GDP,Target
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,17,5,171,1,1,122.0,1,19,12,⋯,0,0,0,0,0.00000,0,10.8,1.4,1.74,Dropout
1,15,1,9254,1,1,160.0,1,1,3,⋯,0,6,6,6,13.66667,0,13.9,-0.3,0.79,Graduate
1,1,5,9070,1,1,122.0,1,37,37,⋯,0,6,0,0,0.00000,0,10.8,1.4,1.74,Dropout
1,17,2,9773,1,1,122.0,1,38,37,⋯,0,6,10,5,12.40000,0,9.4,-0.8,-3.12,Graduate
2,39,1,8014,0,1,100.0,1,37,38,⋯,0,6,6,6,13.00000,0,13.9,-0.3,0.79,Graduate
2,39,1,9991,0,19,133.1,1,37,37,⋯,0,5,17,5,11.50000,5,16.2,0.3,-0.92,Graduate


In [3]:
tidy_data <- data|>
select(Previous_qualification, Previous_qualification_grade, Admission_grade, Target) |> # select our columns of interest
filter(Previous_qualification == 1) |> #We only want to examine observations whose previous education is completed secondary education
filter(Target == "Graduate" | Target == "Dropout") |> #We only want to examine the binary of dropout vs graduate (not students currently enrolled)
mutate(Target = as_factor(Target)) #We want our classification variable to be a factor as there's a limited amount of categories
head(tidy_data)
nrow(tidy_data) #Check we still have a sufficient amount of observations that meet our criteria

Previous_qualification,Previous_qualification_grade,Admission_grade,Target
<dbl>,<dbl>,<dbl>,<fct>
1,122,127.3,Dropout
1,160,142.5,Graduate
1,122,124.8,Dropout
1,122,119.6,Graduate
1,100,141.5,Graduate
1,142,128.4,Graduate


[1] 3019

In [4]:
#Split the data into training and testing. 75% for training and 25% for testing.
# Because we are classifying target, use that as strata
# Set seed so analysis is reproducible
set.seed(2004)

data_split <- initial_split(tidy_data, prop = 0.75, strata = Target) 
dropout_training <- training(data_split) #name training and testing data sets
dropout_testing <- testing(data_split)

In [8]:
set.seed(2000)

# Create Recipe
dropout_recipe <- recipe(Target ~ Previous_qualification_grade + Admission_grade, data = dropout_training) |>
step_upsample(Target, over_ratio = 1, skip = FALSE) |>
step_scale(allpredictors()) |>
step_center(allpredictors())
prep()

#Specify folds for cross validation
dropout_vfold <- vfold_cv(dropout_training, v = 8, strata = Target)

#Specify K-values to try
gridvals <- tibble("neighbors" = seq(from = 1, to = 10, by = 1))

#Add specification
dropout_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
set_engine("kknn") |>
set_mode("classification")

dropout_workflow <- workflow() |>
add_recipe(dropout_recipe) |>
add_model(dropout_spec) |>
tune_grid(resamples = dropout_vfold, grid = gridvals) |>
collect_metrics() |>
filter(.metric == "accuracy")

dropout_workflow

cross_val_plot <- dropout_workflow |>
ggplot(aes(x = neighbors, y = mean)) + 
geom_point() +
geom_line() + 
xlab("Number of Neighbors") + 
ylab("Accuracy") +
ggtitle("K vs Accuracy")

cross_val_plot

ERROR: Error in step_upsample(recipe(Target ~ Previous_qualification_grade + : could not find function "step_upsample"


In [ ]:
cross_val_plot <- dropout_workflow |>
ggplot(aes(x = neighbors, y = mean)) + 
geom_point() +
geom_line() + 
xlab("Number of Neighbors") + 
ylab("Accuracy") +
ggtitle("K vs Accuracy")

cross_val_plot